In [1]:
import os
from docx import Document
import pandas as pd

import pymorphy2
import string
import nltk
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from pymystem3 import Mystem

In [2]:
def get_docx_paragraph_texts(document):
    lst_lines = []
    dict_paragraphs_texts = {}
    for line, p in enumerate(document.paragraphs()):
        p.paragraph_format
        text_line = p.text.replace('\n', ' ').strip()
        if len(text_line) > 0:
            text_line= text_line + '.'
            text_line = text_line.strip().lstrip().rstrip().replace("..", ".").replace("	", " ").replace(" .", ".")
        lst_lines.append("Paragraph {}: ".format(line)+text_line)
        dict_paragraphs_texts[line] = text_line
    return lst_lines, dict_paragraphs_texts

In [3]:
def get_docx_sections_texts(document):
    dict_sections_texts = {}
    paragraphs = document.paragraphs
    section_name = paragraphs[0].text
    dict_sections_texts[section_name] = []
    n = 1
    for line, p in enumerate(paragraphs[1:]):
        section_alignment = str(p.paragraph_format.alignment).split()[0] 
        section_text = p.text
        #print(n, section_alignment, section_text)
        if section_alignment == 'CENTER':
            section_name = section_text
            dict_sections_texts[section_name] = []

        elif section_alignment != 'CENTER':
            dict_sections_texts[section_name].append(section_text)
            
        n = n+1
    return dict_sections_texts

In [4]:
path_docs = "/Users/ulananazarenko/Desktop/Диплом/ВКР_2020/"
lst_paths_files = []
for root, dirs, files in os.walk(path_docs):
    for name in files:
        path_file = os.path.join(root, name)
        lst_paths_files.append(path_file)

In [5]:
lst_dict_sections_texts = []
lst_papers = []
lst_paths_docs = []
for path_file in lst_paths_files:
    path_file = path_file.replace('\\', '/')
    #print(path_file)
    #try:
    f = open(path_file, 'rb')
    document = Document(f)
    paper = [p.text for p in document.paragraphs]
    lst_papers.append(paper)
    
    lst_dict_sections_texts.append(get_docx_sections_texts(document))
    lst_paths_docs.append(path_file)
    #print('\n')
    #except:
     #   print("ERROR:", path_file)
      #  print('\n')
       # continue

BadZipFile: File is not a zip file

In [68]:
df_docs = pd.DataFrame()
df_docs['path_doc'] = lst_paths_docs
df_docs['title'] = df_docs['path_doc'].apply(lambda x: x.split('/')[-1])
df_docs['text_paper'] = lst_papers

df_docs['dict_sections_texts'] = lst_dict_sections_texts

df_docs['faculty_department'] = df_docs['path_doc'].apply(lambda x: x.replace(path_docs, '').split('/')[:-1])
df_docs['faculty'] = df_docs['faculty_department'].apply(lambda x: x[0])
df_docs['department'] = df_docs['faculty_department'].apply(lambda x: x[1] if len(x)>1 else x[0])

df_docs['degree'] = ['bachelor' for _ in range(len(df_docs))]
print(len(df_docs))
df_docs.head()

0


,path_doc,title,text_paper,dict_sections_texts,faculty_department,faculty,department,degree


In [69]:
import string

def simple_dict_sections_texts_cleaning(dict_sections_texts):
    dict_sections_texts_new = {}
    for section, texts in dict_sections_texts.items():
        if len(texts) == 0:
            continue
        section = section.lower().replace('\t', ' ').replace('\n', ' ').replace('  ', ' ').strip()
        section = ''.join([ch for ch in section if ch not in string.punctuation])
        dict_sections_texts_new[section] = []
        for text in texts:
            text = text.replace('\t', ' ').replace('\n', ' ').replace('  ', ' ').strip()
            if text != '':
                dict_sections_texts_new[section].append(text)
        if len(dict_sections_texts_new[section]) == 0:
            dict_sections_texts_new.pop(section)
    return dict_sections_texts_new


def get_main_clean_text(dict_sections_texts):
    lst_texts = []
    for section, lst in dict_sections_texts.items():
        if len(lst)>0:
            lst_texts = lst_texts+lst
    return lst_texts


df_docs['dict_sections_texts_clean'] = df_docs['dict_sections_texts'].map(simple_dict_sections_texts_cleaning)
df_docs['main_clean_text'] = df_docs['dict_sections_texts_clean'].map(get_main_clean_text)
df_docs.head()

,path_doc,title,text_paper,dict_sections_texts,faculty_department,faculty,department,degree,dict_sections_texts_clean,main_clean_text


In [8]:
russian_stopwords = stopwords.words("russian")
russian_stopwords = set(russian_stopwords + ['который', 'таблица', 'рисунок', 
                                             'тот', 'также', 'этот', 'это', 
                                             'такой', 'каждый', 'другой'])

morph = pymorphy2.MorphAnalyzer()

def sentences_preproc(lst_sentences):
    lst_sentences_new = []
    for sentence in list(lst_sentences):
        sentence = ''.join([ch for ch in sentence if ch not in string.punctuation])
        sentence = sentence.split()
        
        sentence_new = [morph.parse(word)[0].normal_form for word in sentence if word not in russian_stopwords and \
                    str(morph.parse(word)[0].tag) != 'UNKN']
        
        if len(sentence_new) > 1:
            lst_sentences_new.append(' '.join(sentence_new))
    return lst_sentences_new

df_docs['preproc_text'] = df_docs['main_clean_text'].map(sentences_preproc)
df_docs.head()

,path_doc,title,text_paper,dict_sections_texts,faculty_department,faculty,department,degree,dict_sections_texts_clean,main_clean_text,preproc_text
0,C:/Users/JM 505 Computers/Desktop/Repetitor/Ул...,Анализ методов обеспечения безопасной передачи...,"[ВЫПУСКНАЯ КВАЛИФИКАЦИОННАЯ РАБОТА, АНАЛИЗ МЕТ...","{'ВЫПУСКНАЯ КВАЛИФИКАЦИОННАЯ РАБОТА': [], 'АНА...",[ФБИТ],ФБИТ,ФБИТ,bachelor,{'анализ методов обеспечения безопасной переда...,"[Автор Павлов Денис Дмитриевич, (Фамилия, Имя,...","[автор павлов денис дмитриевич, фамилия имя от..."
1,C:/Users/JM 505 Computers/Desktop/Repetitor/Ул...,Анализ психологических типов сотрудников для м...,[Министерство науки и высшего образования Росс...,{'Министерство науки и высшего образования Рос...,[ФБИТ],ФБИТ,ФБИТ,bachelor,{'министерство науки и высшего образования рос...,[ФЕДЕРАЛЬНОЕ ГОСУДАРСТВЕННОЕ АВТОНОМНОЕ ОБРАЗО...,[федеральный государственный автономный образо...
2,C:/Users/JM 505 Computers/Desktop/Repetitor/Ул...,Исследование влияния алгоритма стохастического...,[Министерство науки и высшего образования Росс...,{'Министерство науки и высшего образования Рос...,[ФБИТ],ФБИТ,ФБИТ,bachelor,{'исследование влияния алгоритма стохастическо...,"[Автор Бахтиярова Алина Шамильевна (Фамилия, И...",[автор бахтияров алина шамильевич фамилия имя ...
3,C:/Users/JM 505 Computers/Desktop/Repetitor/Ул...,Разработка VPN-приложения для мобильных устрой...,[Министерство науки и высшего образования Росс...,{'Министерство науки и высшего образования Рос...,[ФБИТ],ФБИТ,ФБИТ,bachelor,{'выпускная квалификационная работа': ['РАЗРАБ...,[РАЗРАБОТКА VPN-ПРИЛОЖЕНИЯ ДЛЯ МОБИЛЬНЫХ УСТРО...,[разработка vpnприложение для мобильный устрой...
4,C:/Users/JM 505 Computers/Desktop/Repetitor/Ул...,Разработка автоматизированной системы защиты и...,[Министерство науки и высшего образования Росс...,{'Министерство науки и высшего образования Рос...,[ФБИТ],ФБИТ,ФБИТ,bachelor,{'разработка автоматизированной системы защиты...,"[Автор Пенин Андрей Семенович, (Фамилия, Имя, ...","[автор пенина андрей семёнович, фамилия имя от..."


In [11]:
df_docs.to_csv('VKR_docs.csv', index = False)
df_docs.to_excel('VKR_docs.xlsx', index = False)